In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator
from sklearn.datasets import make_regression
from sklearn.datasets import make_friedman1

Вставьте код в класс, приведённый ниже. В итоге должна получиться композиция деревьев

In [19]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters=100, tau=1e-1):
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        self.estimators = []
        curr_pred = 0
        for iter_num in range(self.iters):
            algo = DecisionTreeRegressor(
                max_depth=self.tree_params_dict.get('max_depth', 10), 
                min_samples_leaf = self.tree_params_dict.get('min_samples_leaf', 2)
            )
            algo.fit(X_data, 2*(y_data - self.predict(X_data)))
            self.estimators.append(algo)
    
    def predict(self, X_data):
        res = np.zeros(X_data.shape[0])
        for estimator in self.estimators:
            res += self.tau * estimator.predict(X_data)
        return res

## Проверка качества полученного класса

Можете поиграться с параметрами, посмотрим, у кого самое лучшее качество получится

Сгенерируем данные с линейной зависимостью

In [31]:
X_data, y_data = make_regression(n_samples=1000, noise=10, n_features=10, random_state=42)

In [64]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth':4,
        'min_samples_leaf':5
    },
    iters=500,
    tau = 0.1
)

In [65]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-122.46529985862792

In [4]:
np.mean(cross_val_score(DecisionTreeRegressor(max_depth=4), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-7550.7275207710281

In [5]:
np.mean(cross_val_score(LinearRegression(), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


-95.517114056519944

на реально-линейной зависимости не дотягивает до линейной модели, но уже существенно лучше чем одно дерево.

И аналогично, сгенерируем нелинейную зависимость

In [40]:
X_data, y_data = make_friedman1(n_samples=1000, noise=10, n_features=10, random_state=42)

In [42]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth': 1,
        'min_samples_leaf': 2
    },
    iters=100,
    tau=0.1
)

In [43]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-106.29139653138145

In [7]:
np.mean(cross_val_score(DecisionTreeRegressor(max_depth=6), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-139.32261423591237

In [8]:
np.mean(cross_val_score(LinearRegression(), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-108.80318041227227

### Финальный код оценки качества (вам нужно GB реализовать и подобрать хорошие tree_params)

In [82]:
tree_params_dict = {
    'max_depth': 1,
    'min_samples_leaf': 6
} # TODO
iters = 100 #TODO
tau = 0.07 # TODO

algo = SimpleGB(
    tree_params_dict=tree_params_dict,
    iters=iters,
    tau=tau
)
print(np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='neg_mean_squared_error')))

-105.758472419
